In [ ]:
              # DataSet : Kenya_1981_2023_detrended_var

In [ ]:
import pandas as pd
from scipy import stats
import numpy as np
import scipy
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
import datetime
import plotly
import plotly.graph_objs as go
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly.express as px
import warnings
warnings.filterwarnings('ignore')

In [ ]:
import statsmodels.formula.api as sm
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import FunctionTransformer
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
from sklearn import tree
from sklearn.model_selection import cross_val_score

In [ ]:
#!pip install catboost

In [ ]:
# Import Regressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import LinearRegression,Lasso,Ridge,ElasticNet
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import AdaBoostRegressor
from sklearn.svm import SVR
import xgboost as xgb
from sklearn.linear_model import BayesianRidge
from sklearn.neighbors import KNeighborsRegressor
from sklearn.ensemble import ExtraTreesRegressor
# from catboost import CatBoostRegressor
import lightgbm as lgb

#import the metrics
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

ModuleNotFoundError: No module named 'catboost'

In [ ]:
df = pd.read_csv('Kenya_1981_2023_detrended_var.csv')
df.head()

In [ ]:
df.info()

In [ ]:
df.shape

In [ ]:
# Value counts for each categorical feature:

for column in df.select_dtypes(include=['object']):
    print(f"Value counts for {column}:")
    print(df[column].value_counts())
    print(df[column].nunique())
    print("="*45)

In [ ]:
# Check for missing values
missing_values = df.isnull().sum()
missing_values

In [ ]:
df = df.drop(columns=["Unnamed: 0", "area"])

# Univariate Analysis - Histogram

In [ ]:
for i in df.columns:
    plt.figure(figsize = (10,4))
    sns.histplot(df[i], kde=True)
    plt.xlabel(i)
    plt.show()

In [ ]:
# Calculate skewness for all numerical columns
skewness = df.select_dtypes(include=['number']).skew()

# Display the skewness values
print(skewness)

# Outlier Detection

In [ ]:
# Detecting outliers using IQR (Interquartile Range)
numeric_data = df.select_dtypes(include=['number'])

# Calculate the first and third quartile
Q1 = numeric_data.quantile(0.25)
Q3 = numeric_data.quantile(0.75)

# Calculate the interquartile range (IQR)
IQR = Q3 - Q1

# Detect outliers
outliers = ((numeric_data < (Q1 - 1.5 * IQR)) | (numeric_data > (Q3 + 1.5 * IQR))).sum()

print(outliers)

In [ ]:
print(df.select_dtypes(include=['int64','float64']).columns)

# Insight into 'yield_fao_1000ha' - Histogram Plot

In [ ]:
plt.figure(figsize=(6, 4))
sns.histplot(data=df, x='yield_fao_1000ha', element='step', common_norm=False)


mean = np.mean(df['yield_fao_1000ha'])
median = np.median(df['yield_fao_1000ha'])
std_deviation = np.std(df['yield_fao_1000ha'])

plt.axvline(mean, color='red', linestyle='--')
plt.axvline(median, color='blue', linestyle='--')
plt.axvline(x=std_deviation, color='green', linestyle='--')

# annotate mean,median & Standard deviation
plt.annotate(f'Mean: {mean:.3}', xy=(mean, plt.gca().get_ylim()[1]), xytext=(85, -30),
             textcoords='offset points', color='red', fontsize=10)
plt.annotate(f'Median: {median:.3f}', xy=(median, plt.gca().get_ylim()[1]), xytext=(90, -50),
             textcoords='offset points', color='blue', fontsize=10)
plt.annotate(f'Standard deviation: {std_deviation:.3f}', xy=(std_deviation, plt.gca().get_ylim()[1]), xytext=(-50, -70),
           textcoords='offset points', color='green', fontsize=10)
plt.xlabel('yield_fao_1000ha')
plt.ylabel('Frequency')
plt.title('Distribution of yield_fao_1000ha')
plt.show()

# Insight into 'yield_usda_1000ha' - Histogram Plot

In [ ]:
plt.figure(figsize=(6, 4))
sns.histplot(data=df, x='yield_usda_1000ha', element='step', common_norm=False)


mean = np.mean(df['yield_usda_1000ha'])
median = np.median(df['yield_usda_1000ha'])
std_deviation = np.std(df['yield_usda_1000ha'])

plt.axvline(mean, color='red', linestyle='--')
plt.axvline(median, color='blue', linestyle='--')
plt.axvline(x=std_deviation, color='green', linestyle='--')

# annotate mean,median & Standard deviation
plt.annotate(f'Mean: {mean:.3}', xy=(mean, plt.gca().get_ylim()[1]), xytext=(-120, -30),
             textcoords='offset points', color='red', fontsize=10)
plt.annotate(f'Median: {median:.3f}', xy=(median, plt.gca().get_ylim()[1]), xytext=(-120, -50),
             textcoords='offset points', color='blue', fontsize=10)
plt.annotate(f'Standard deviation: {std_deviation:.3f}', xy=(std_deviation, plt.gca().get_ylim()[1]), xytext=(-210, -70),
           textcoords='offset points', color='green', fontsize=10)
plt.xlabel('yield_usda_1000ha')
plt.ylabel('Frequency')
plt.title('Distribution of yield_usda_1000ha')
plt.show()

# Correlation Analysis

In [ ]:
df_corr = df.corr(method='pearson')
df_corr= df_corr.applymap("{:.3f}".format)
df_corr

In [ ]:
# Correlation Matrix
correlation_matrix = df.corr()
plt.figure(figsize=(18, 16))
mask = np.triu(correlation_matrix)
sns.heatmap(correlation_matrix, annot=True, cmap='viridis_r', center=0, mask=mask, fmt=".2f")
plt.xticks(rotation = 90, fontsize=12)
plt.yticks(fontsize=12)
plt.title('Correlational Heatmap')
plt.show()

In [ ]:
df_corr["yield_usda_1000ha"].sort_values(ascending=False)

In [ ]:
df_corr["yield_fao_1000ha"].sort_values(ascending=False)

# 1) Modeling - Target: "yield_fao_1000ha" - All Features

In [ ]:
df.columns

In [ ]:
# columns to exclude : 'year' , 'yield_usda_1000ha'

num_var = ['area_harvested_usda_1000ha', 'production_usda_1000ha',
       'area_harvested_fao_1000ha',
       'production_fao_1000ha', 'dew_temp_C',
       'soil_temp_L1_C', 'soil_temp_L2_C', 'soil_temp_L3_C', 'soil_temp_L4_C',
       'irradiation_J_m2', 'temp_C', 'max_temp_C', 'min_temp_C',
       'evaporation_mm', 'precipitation_era5_mm', 'wind_eastward_m_s',
       'wind_northward_m_s', 'soil_water_L1_fraction',
       'soil_water_L2_fraction', 'soil_water_L3_fraction',
       'soil_water_L4_fraction', 'precipitation_chirps_mm']

cat_var = []

y = df['yield_fao_1000ha']

X = df[cat_var + num_var]
y = pd.DataFrame(y, columns=['yield_fao_1000ha'])

In [ ]:
print(type(X))
print(type(y))

In [ ]:
X.info()

In [ ]:
# Train-test split:

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42 , shuffle=True)

print(f"X_train.shape: {X_train.shape}")
print(f"X_test.shape: {X_test.shape}")
print(f"y_train.shape: {y_train.shape}")
print(f"y_test.shape: {y_test.shape}")

In [ ]:
# Preprocessing with pipelines

from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import MinMaxScaler, OneHotEncoder


pipeline_categorical = Pipeline([('onehot', OneHotEncoder(handle_unknown="ignore")),])

pipeline_numerical = Pipeline([('scaler', MinMaxScaler(feature_range=(0,1))),])

pipeline_full = ColumnTransformer([("categorical", pipeline_categorical, cat_var),
                                   ("numerical", pipeline_numerical, num_var)])

In [ ]:
# Apply the pipeline

pipeline_full.fit(X_train)

X_train_transformed = pipeline_full.transform(X_train)
X_test_transformed = pipeline_full.transform(X_test)

print(f"X_train_transformed.shape: {X_train_transformed.shape}")
print(f"X_test_transformed.shape: {X_test_transformed.shape}")
print()
print('X_train_transformed:', type(X_train_transformed))
print('X_test_transformed:', type(X_test_transformed))

In [ ]:
models = {
    'LinearRegression': LinearRegression(),
    'LassoRegressor': Lasso(max_iter=10000, random_state=42),
    'RidgeRegressor': Ridge(max_iter=10000, random_state=42),
    "ElasticNet": ElasticNet(alpha=0.02, random_state=42),
    # "KNN": KNeighborsRegressor(),
    'DTR': DecisionTreeRegressor(random_state=42),
    'SVR': SVR(),
    'RFR': RandomForestRegressor(random_state=42),
    "ExtraTrees": ExtraTreesRegressor(n_estimators=5, random_state=42),
    'ADABoost': AdaBoostRegressor(random_state=42),
    'XGBoost': xgb.XGBRegressor(random_state=42),
    'CatBoost': CatBoostRegressor(random_state=42),
    'LightGBM': lgb.LGBMRegressor(random_state=42)
}

In [ ]:
# Train multiple models

model_list=[]
r2_list=[]
mae_list=[]
mse_list=[]
rmse_list=[]

for i in range(len(list(models))):
    model=list(models.values())[i]
    model.fit(X_train_transformed,y_train)

    #Make Predictions
    y_pred=model.predict(X_test_transformed)

    # Evaluation Metric:
    #r2 = model.score(X_test_transformed, y_test)
    r2 = r2_score(y_test, y_pred)
    mae = mean_absolute_error(y_test, y_pred)
    mse = mean_squared_error(y_test, y_pred)
    rmse= np.sqrt(mean_squared_error(y_test, y_pred))

    model_list.append(list(models.keys())[i])
    r2_list.append(r2)
    mae_list.append(mae)
    mse_list.append(mse)
    rmse_list.append(rmse)

    # Print the performance metrics for each model
    print(f"Model: {list(models.keys())[i]}")
    print(f"R2 Score: {r2}")
    print(f"MAE: {mae}")
    print(f"MSE: {mse}")
    print(f"RMSE: {rmse}")
    print("="*30)

In [ ]:
# Find the index of the model with the best r2 score
best_index_r2 = r2_list.index(max(r2_list))

# Find the index of the model with the best mae score
best_index_mae = mae_list.index(min(mae_list))

# Find the index of the model with the best mse score
best_index_mse = mse_list.index(min(mse_list))

# Find the index of the model with the best rmse score
best_index_rmse = rmse_list.index(min(rmse_list))

# Determine the model with the best performance based on the chosen metric
best_r2_model = model_list[best_index_r2]
best_mae_model = model_list[best_index_mae]
best_mse_model = model_list[best_index_mse]
best_rmse_model = model_list[best_index_rmse]

# Print the best models and their corresponding metric
print("The model with the best R2 score is:", best_r2_model)
print("R2 score:", r2_list[best_index_r2])
print("="*40)
print("The model with the best MAE score is:", best_mae_model)
print("MAE:", mae_list[best_index_mae])
print("="*40)
print("The model with the best MSE score is:", best_mse_model)
print("MSE:", mse_list[best_index_mse])
print("="*40)
print("The model with the best RMSE score is:", best_rmse_model)
print("RMSE:", rmse_list[best_index_rmse])

In [ ]:
# Create a DF for Result:

results_df = pd.DataFrame({
    'Model': model_list,
    'R2 Score': r2_list,
    'MAE': mae_list,
    'MSE': mse_list,
    'RMSE': rmse_list})

results_df.sort_values(by="RMSE", ascending=True)

# **First Top Model: Ridge Regressor**

**Hyperparameter Search for Ridge Regression:**

In [ ]:
# Hyperparameter Search for Ridge Regression:

from sklearn.linear_model import Ridge
from sklearn.model_selection import GridSearchCV

param_grid = {
    'alpha': [0.01, 0.1, 1.0, 10.0],
    'solver': ['auto', 'svd', 'cholesky', 'lsqr', 'sparse_cg', 'sag', 'saga']}

grid_search = GridSearchCV(
    estimator=Ridge(),
    param_grid=param_grid,
    scoring='neg_mean_squared_error',
    cv=5, n_jobs=-1, verbose=1)

grid_search.fit(X_train_transformed, y_train)


In [ ]:
print(grid_search.best_params_)
print("\n",grid_search.best_estimator_)

In [ ]:
Tuned_Ridge_Reg = grid_search.best_estimator_
Tuned_Ridge_Reg

In [ ]:
Tuned_Ridge_Reg .fit(X_train_transformed,y_train)
y_pred=Tuned_Ridge_Reg.predict(X_test_transformed)

In [ ]:
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error

r2 = r2_score(y_test, y_pred)
mae = mean_absolute_error(y_test, y_pred)
mse = mean_squared_error(y_test, y_pred)
rmse = np.sqrt(mse)

print(f"R2 Score: {r2}")
print(f"MAE: {mse}")
print(f"MSE: {mae}")
print(f"RMSE: {rmse}")

In [ ]:
# Calculate residuals
residuals = y_test.values.flatten() - y_pred.flatten()

# Visualize residuals
#sns.kdeplot(residuals)
sns.histplot(residuals, kde=True)
plt.title('Distribution of Residuals - Tuned Ridge Regression')
plt.xlabel('Residuals')
plt.ylabel('Density')
plt.show()

In [ ]:
# Line Plot: Actual vs Predicted Values

plt.plot([y_test.min(), y_test.max()], [y_test.min(), y_test.max()], '--', color='green', label='Actual')
plt.scatter(y_test, y_pred, label='Tuned Ridge Regression Predictions')
plt.xlabel('Actual Values')
plt.ylabel('Predicted Values')
plt.title('Actual vs Predicted Values - Tuned Ridge Regression')
plt.show()

**Ploting the Learning Curve**

In [ ]:
# Ploting the learning curve:

from sklearn.model_selection import learning_curve
train_sizes, train_scores, test_scores = learning_curve(estimator=Tuned_Ridge_Reg, X=X_train_transformed, y=y_train,
                                                       cv=5, train_sizes=np.linspace(0.1, 1.0, 20), n_jobs=-1)

plt.figure(figsize=(8,4))
# Calculate training and test mean and std

train_mean = np.mean(train_scores, axis=1)
train_std = np.std(train_scores, axis=1)
test_mean = np.mean(test_scores, axis=1)
test_std = np.std(test_scores, axis=1)

# Plot the learning curve

plt.plot(train_sizes, train_mean, color='blue', marker='o', markersize=5, label='Training Score')
plt.fill_between(train_sizes, train_mean + train_std, train_mean - train_std, alpha=0.15, color='blue')

plt.plot(train_sizes, test_mean, color='green', marker='o', markersize=5, label='Cross-Validation Score')
plt.fill_between(train_sizes, test_mean + test_std, test_mean - test_std, alpha=0.15, color='green')

plt.title('Tuned Ridge Regression Learning Curve ')
plt.xlabel('Training Example', fontsize=12)
plt.ylabel('Accuracy Score', fontsize=12)

plt.legend(loc='lower center', fontsize=8)
plt.show()


# Forward Feature Selection

In [ ]:
from mlxtend.feature_selection import SequentialFeatureSelector as SFS

# Perform forward feature selection
sfs_forward = SFS(Tuned_Ridge_Reg,
                  k_features='best',
                  forward=True,
                  floating=False,
                  scoring='neg_mean_squared_error',
                  cv=5)

sfs_forward.fit(X_train_transformed, y_train.values.ravel())

# Selected features using forward selection
selected_features_forward_indices = sfs_forward.k_feature_idx_
selected_features_forward_names = [num_var[i] for i in selected_features_forward_indices]
print(f'Selected features (Forward): {selected_features_forward_names}')

# Modeling - Target : 'yield_fao_1000ha' - Selected Features

In [ ]:
num_var = ['area_harvested_usda_1000ha',
 'area_harvested_fao_1000ha',
 'production_fao_1000ha',
 'soil_temp_L3_C',
 'temp_C',
 'max_temp_C',
 'min_temp_C',
 'evaporation_mm',
 'wind_eastward_m_s',
 'soil_water_L1_fraction',
 'precipitation_chirps_mm']


cat_var = []

y = df['yield_fao_1000ha']

X = df[cat_var + num_var]
y = pd.DataFrame(y, columns=['yield_fao_1000ha'])

In [ ]:
X.info()

In [ ]:
# Train-test split:

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42 , shuffle=True)

print(f"X_train.shape: {X_train.shape}")
print(f"X_test.shape: {X_test.shape}")
print(f"y_train.shape: {y_train.shape}")
print(f"y_test.shape: {y_test.shape}")

In [ ]:
# Preprocessing with pipelines

from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import MinMaxScaler, OneHotEncoder


pipeline_categorical = Pipeline([('onehot', OneHotEncoder(handle_unknown="ignore")),])

pipeline_numerical = Pipeline([('scaler', MinMaxScaler(feature_range=(0,1))),])

pipeline_full = ColumnTransformer([("categorical", pipeline_categorical, cat_var),
                                   ("numerical", pipeline_numerical, num_var)])

In [ ]:
# Apply the pipeline

pipeline_full.fit(X_train)

X_train_transformed = pipeline_full.transform(X_train)
X_test_transformed = pipeline_full.transform(X_test)

print(f"X_train_transformed.shape: {X_train_transformed.shape}")
print(f"X_test_transformed.shape: {X_test_transformed.shape}")
print()
print('X_train_transformed:', type(X_train_transformed))
print('X_test_transformed:', type(X_test_transformed))

In [ ]:
models = {
    'LinearRegression': LinearRegression(),
    'LassoRegressor': Lasso(max_iter=10000, random_state=42),
    'RidgeRegressor': Ridge(max_iter=10000, random_state=42),
    "ElasticNet": ElasticNet(alpha=0.02, random_state=42),
    # "KNN": KNeighborsRegressor(),
    'DTR': DecisionTreeRegressor(random_state=42),
    'SVR': SVR(),
    'RFR': RandomForestRegressor(random_state=42),
    "ExtraTrees": ExtraTreesRegressor(n_estimators=5, random_state=42),
    'ADABoost': AdaBoostRegressor(random_state=42),
    'XGBoost': xgb.XGBRegressor(random_state=42),
    'CatBoost': CatBoostRegressor(random_state=42),
    'LightGBM': lgb.LGBMRegressor(random_state=42)
}

In [ ]:
# Train multiple models

model_list=[]
r2_list=[]
mae_list=[]
mse_list=[]
rmse_list=[]

for i in range(len(list(models))):
    model=list(models.values())[i]
    model.fit(X_train_transformed,y_train)

    #Make Predictions
    y_pred=model.predict(X_test_transformed)

    # Evaluation Metric:
    #r2 = model.score(X_test_transformed, y_test)
    r2 = r2_score(y_test, y_pred)
    mae = mean_absolute_error(y_test, y_pred)
    mse = mean_squared_error(y_test, y_pred)
    rmse= np.sqrt(mean_squared_error(y_test, y_pred))

    model_list.append(list(models.keys())[i])
    r2_list.append(r2)
    mae_list.append(mae)
    mse_list.append(mse)
    rmse_list.append(rmse)

    # Print the performance metrics for each model
    print(f"Model: {list(models.keys())[i]}")
    print(f"R2 Score: {r2}")
    print(f"MAE: {mae}")
    print(f"MSE: {mse}")
    print(f"RMSE: {rmse}")
    print("="*30)

In [ ]:
# Find the index of the model with the best r2 score
best_index_r2 = r2_list.index(max(r2_list))

# Find the index of the model with the best mae score
best_index_mae = mae_list.index(min(mae_list))

# Find the index of the model with the best mse score
best_index_mse = mse_list.index(min(mse_list))

# Find the index of the model with the best rmse score
best_index_rmse = rmse_list.index(min(rmse_list))

# Determine the model with the best performance based on the chosen metric
best_r2_model = model_list[best_index_r2]
best_mae_model = model_list[best_index_mae]
best_mse_model = model_list[best_index_mse]
best_rmse_model = model_list[best_index_rmse]

# Print the best models and their corresponding metric
print("The model with the best R2 score is:", best_r2_model)
print("R2 score:", r2_list[best_index_r2])
print("="*40)
print("The model with the best MAE score is:", best_mae_model)
print("MAE:", mae_list[best_index_mae])
print("="*40)
print("The model with the best MSE score is:", best_mse_model)
print("MSE:", mse_list[best_index_mse])
print("="*40)
print("The model with the best RMSE score is:", best_rmse_model)
print("RMSE:", rmse_list[best_index_rmse])

In [ ]:
# Create a DF for Result:

results_df = pd.DataFrame({
    'Model': model_list,
    'R2 Score': r2_list,
    'MAE': mae_list,
    'MSE': mse_list,
    'RMSE': rmse_list})

results_df.sort_values(by="RMSE", ascending=True)

# First Top Model: Ridge Regressor

**Hyperparameter Search for Ridge Regression:**

In [ ]:
# Hyperparameter Search for Ridge Regression:

from sklearn.linear_model import Ridge
from sklearn.model_selection import GridSearchCV

param_grid = {
    'alpha': [0.01, 0.1, 1.0, 10.0],
    'solver': ['auto', 'svd', 'cholesky', 'lsqr', 'sparse_cg', 'sag', 'saga']}

grid_search = GridSearchCV(
    estimator=Ridge(),
    param_grid=param_grid,
    scoring='neg_mean_squared_error',
    cv=5, n_jobs=-1, verbose=1)

grid_search.fit(X_train_transformed, y_train)


In [ ]:
print(grid_search.best_params_)
print("\n",grid_search.best_estimator_)

In [ ]:
Tuned_Ridge_Reg = grid_search.best_estimator_
Tuned_Ridge_Reg

In [ ]:
Tuned_Ridge_Reg .fit(X_train_transformed,y_train)
y_pred=Tuned_Ridge_Reg.predict(X_test_transformed)

In [ ]:
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error

r2 = r2_score(y_test, y_pred)
mae = mean_absolute_error(y_test, y_pred)
mse = mean_squared_error(y_test, y_pred)
rmse = np.sqrt(mse)

print(f"R2 Score: {r2}")
print(f"MAE: {mse}")
print(f"MSE: {mae}")
print(f"RMSE: {rmse}")

In [ ]:
# Calculate residuals
residuals = y_test.values.flatten() - y_pred.flatten()

# Visualize residuals
#sns.kdeplot(residuals)
sns.histplot(residuals, kde=True)
plt.title('Distribution of Residuals - Tuned Ridge Regression')
plt.xlabel('Residuals')
plt.ylabel('Density')
plt.show()

In [ ]:
# Line Plot: Actual vs Predicted Values

plt.plot([y_test.min(), y_test.max()], [y_test.min(), y_test.max()], '--', color='green', label='Actual')
plt.scatter(y_test, y_pred, label='Tuned Ridge Regression Predictions')
plt.xlabel('Actual Values')
plt.ylabel('Predicted Values')
plt.title('Actual vs Predicted Values - Tuned Ridge Regression')
plt.show()

In [ ]:
# Ploting the learning curve:

from sklearn.model_selection import learning_curve
train_sizes, train_scores, test_scores = learning_curve(estimator=Tuned_Ridge_Reg, X=X_train_transformed, y=y_train,
                                                       cv=5, train_sizes=np.linspace(0.1, 1.0, 20), n_jobs=-1)

plt.figure(figsize=(8,4))
# Calculate training and test mean and std

train_mean = np.mean(train_scores, axis=1)
train_std = np.std(train_scores, axis=1)
test_mean = np.mean(test_scores, axis=1)
test_std = np.std(test_scores, axis=1)

# Plot the learning curve

plt.plot(train_sizes, train_mean, color='blue', marker='o', markersize=5, label='Training Score')
plt.fill_between(train_sizes, train_mean + train_std, train_mean - train_std, alpha=0.15, color='blue')

plt.plot(train_sizes, test_mean, color='green', marker='o', markersize=5, label='Cross-Validation Score')
plt.fill_between(train_sizes, test_mean + test_std, test_mean - test_std, alpha=0.15, color='green')

plt.title('Tuned Ridge Regression Learning Curve ')
plt.xlabel('Training Example', fontsize=12)
plt.ylabel('Accuracy Score', fontsize=12)

plt.legend(loc='lower center', fontsize=8)
plt.show()


# 2) Modeling - Target: "yield_usda_1000ha" - All Features

In [ ]:
df.columns

In [ ]:
# Columns to exclude = "year", 'yield_fao_1000ha'

num_var = ['area_harvested_usda_1000ha', 'production_usda_1000ha',
        'area_harvested_fao_1000ha',
       'production_fao_1000ha', 'dew_temp_C',
       'soil_temp_L1_C', 'soil_temp_L2_C', 'soil_temp_L3_C', 'soil_temp_L4_C',
       'irradiation_J_m2', 'temp_C', 'max_temp_C', 'min_temp_C',
       'evaporation_mm', 'precipitation_era5_mm', 'wind_eastward_m_s',
       'wind_northward_m_s', 'soil_water_L1_fraction',
       'soil_water_L2_fraction', 'soil_water_L3_fraction',
       'soil_water_L4_fraction', 'precipitation_chirps_mm']

cat_var = []

y = df['yield_usda_1000ha']

X = df[cat_var + num_var]
y = pd.DataFrame(y, columns=['yield_usda_1000ha'])

In [ ]:
print(type(X))
print(type(y))

In [ ]:
X.info()

In [ ]:
# Train-test split:

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42 , shuffle=True)

print(f"X_train.shape: {X_train.shape}")
print(f"X_test.shape: {X_test.shape}")
print(f"y_train.shape: {y_train.shape}")
print(f"y_test.shape: {y_test.shape}")

In [ ]:
# Preprocessing with pipelines

from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import MinMaxScaler, OneHotEncoder


pipeline_categorical = Pipeline([('onehot', OneHotEncoder(handle_unknown="ignore")),])

pipeline_numerical = Pipeline([('scaler', MinMaxScaler(feature_range=(0,1))),])

pipeline_full = ColumnTransformer([("categorical", pipeline_categorical, cat_var),
                                   ("numerical", pipeline_numerical, num_var)])

In [ ]:
# Apply the pipeline

pipeline_full.fit(X_train)

X_train_transformed = pipeline_full.transform(X_train)
X_test_transformed = pipeline_full.transform(X_test)

print(f"X_train_transformed.shape: {X_train_transformed.shape}")
print(f"X_test_transformed.shape: {X_test_transformed.shape}")
print()
print('X_train_transformed:', type(X_train_transformed))
print('X_test_transformed:', type(X_test_transformed))

In [ ]:
models = {
    'LinearRegression': LinearRegression(),
    'LassoRegressor': Lasso(max_iter=10000, random_state=42),
    'RidgeRegressor': Ridge(max_iter=10000, random_state=42),
    "ElasticNet": ElasticNet(alpha=0.02, random_state=42),
    # "KNN": KNeighborsRegressor(),
    'DTR': DecisionTreeRegressor(random_state=42),
    'SVR': SVR(),
    'RFR': RandomForestRegressor(random_state=42),
    "ExtraTrees": ExtraTreesRegressor(n_estimators=5, random_state=42),
    'ADABoost': AdaBoostRegressor(random_state=42),
    'XGBoost': xgb.XGBRegressor(random_state=42),
    'CatBoost': CatBoostRegressor(random_state=42),
    'LightGBM': lgb.LGBMRegressor(random_state=42)
}

In [ ]:
# Train multiple models

model_list=[]
r2_list=[]
mae_list=[]
mse_list=[]
rmse_list=[]

for i in range(len(list(models))):
    model=list(models.values())[i]
    model.fit(X_train_transformed,y_train)

    #Make Predictions
    y_pred=model.predict(X_test_transformed)

    # Evaluation Metric:
    #r2 = model.score(X_test_transformed, y_test)
    r2 = r2_score(y_test, y_pred)
    mae = mean_absolute_error(y_test, y_pred)
    mse = mean_squared_error(y_test, y_pred)
    rmse= np.sqrt(mean_squared_error(y_test, y_pred))

    model_list.append(list(models.keys())[i])
    r2_list.append(r2)
    mae_list.append(mae)
    mse_list.append(mse)
    rmse_list.append(rmse)

    # Print the performance metrics for each model
    print(f"Model: {list(models.keys())[i]}")
    print(f"R2 Score: {r2}")
    print(f"MAE: {mae}")
    print(f"MSE: {mse}")
    print(f"RMSE: {rmse}")
    print("="*30)

In [ ]:
# Find the index of the model with the best r2 score
best_index_r2 = r2_list.index(max(r2_list))

# Find the index of the model with the best mae score
best_index_mae = mae_list.index(min(mae_list))

# Find the index of the model with the best mse score
best_index_mse = mse_list.index(min(mse_list))

# Find the index of the model with the best rmse score
best_index_rmse = rmse_list.index(min(rmse_list))

# Determine the model with the best performance based on the chosen metric
best_r2_model = model_list[best_index_r2]
best_mae_model = model_list[best_index_mae]
best_mse_model = model_list[best_index_mse]
best_rmse_model = model_list[best_index_rmse]

# Print the best models and their corresponding metric
print("The model with the best R2 score is:", best_r2_model)
print("R2 score:", r2_list[best_index_r2])
print("="*40)
print("The model with the best MAE score is:", best_mae_model)
print("MAE:", mae_list[best_index_mae])
print("="*40)
print("The model with the best MSE score is:", best_mse_model)
print("MSE:", mse_list[best_index_mse])
print("="*40)
print("The model with the best RMSE score is:", best_rmse_model)
print("RMSE:", rmse_list[best_index_rmse])

In [ ]:
# Create a DF for Result:

results_df = pd.DataFrame({
    'Model': model_list,
    'R2 Score': r2_list,
    'MAE': mae_list,
    'MSE': mse_list,
    'RMSE': rmse_list})

results_df.sort_values(by="RMSE", ascending=True)

# Second Top Model: Ridge Regressor

**Hyperparameter Search for Ridge Regression:**

In [ ]:
# Hyperparameter Search for Ridge Regression:

from sklearn.linear_model import Ridge
from sklearn.model_selection import GridSearchCV

param_grid = {
    'alpha': [0.01, 0.1, 1.0, 10.0],
    'solver': ['auto', 'svd', 'cholesky', 'lsqr', 'sparse_cg', 'sag', 'saga']}

grid_search = GridSearchCV(
    estimator=Ridge(),
    param_grid=param_grid,
    scoring='neg_mean_squared_error',
    cv=5, n_jobs=-1, verbose=1)

grid_search.fit(X_train_transformed, y_train)


In [ ]:
print(grid_search.best_params_)
print("\n",grid_search.best_estimator_)

In [ ]:
Tuned_Ridge_Reg = grid_search.best_estimator_
Tuned_Ridge_Reg

In [ ]:
Tuned_Ridge_Reg .fit(X_train_transformed,y_train)
y_pred=Tuned_Ridge_Reg.predict(X_test_transformed)

In [ ]:
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error

r2 = r2_score(y_test, y_pred)
mae = mean_absolute_error(y_test, y_pred)
mse = mean_squared_error(y_test, y_pred)
rmse = np.sqrt(mse)

print(f"R2 Score: {r2}")
print(f"MAE: {mse}")
print(f"MSE: {mae}")
print(f"RMSE: {rmse}")

In [ ]:
# Calculate residuals
residuals = y_test.values.flatten() - y_pred.flatten()

# Visualize residuals
#sns.kdeplot(residuals)
sns.histplot(residuals, kde=True)
plt.title('Distribution of Residuals - Tuned Ridge Regression')
plt.xlabel('Residuals')
plt.ylabel('Density')
plt.show()

In [ ]:
# Line Plot: Actual vs Predicted Values

plt.plot([y_test.min(), y_test.max()], [y_test.min(), y_test.max()], '--', color='green', label='Actual')
plt.scatter(y_test, y_pred, label='Linear Regression Predictions')
plt.xlabel('Actual Values')
plt.ylabel('Predicted Values')
plt.title('Actual vs Predicted Values - Tuned Ridge Regression')
plt.show()

**Ploting the Learning Curve**

In [ ]:
# Ploting the learning curve:

from sklearn.model_selection import learning_curve
train_sizes, train_scores, test_scores = learning_curve(estimator=Tuned_Ridge_Reg,
                                                       X=X_train_transformed, y=y_train,
                                                       cv=5, train_sizes=np.linspace(0.1, 1.0, 20), n_jobs=-1)

plt.figure(figsize=(8,4))
# Calculate training and test mean and std

train_mean = np.mean(train_scores, axis=1)
train_std = np.std(train_scores, axis=1)
test_mean = np.mean(test_scores, axis=1)
test_std = np.std(test_scores, axis=1)

# Plot the learning curve

plt.plot(train_sizes, train_mean, color='blue', marker='o', markersize=5, label='Training Score')
plt.fill_between(train_sizes, train_mean + train_std, train_mean - train_std, alpha=0.15, color='blue')

plt.plot(train_sizes, test_mean, color='green', marker='o', markersize=5, label='Cross-Validation Score')
plt.fill_between(train_sizes, test_mean + test_std, test_mean - test_std, alpha=0.15, color='green')

plt.title('Tuned Ridge Regression Learning Curve ')
plt.xlabel('Training Example', fontsize=12)
plt.ylabel('Accuracy Score', fontsize=12)

plt.legend(loc='lower center', fontsize=8)
plt.show()


# Forward Feature Selection

In [ ]:
from mlxtend.feature_selection import SequentialFeatureSelector as SFS

# Perform forward feature selection
sfs_forward = SFS(Tuned_Ridge_Reg,
                  k_features='best',
                  forward=True,
                  floating=False,
                  scoring='neg_mean_squared_error',
                  cv=5)

sfs_forward.fit(X_train_transformed, y_train.values.ravel())

# Selected features using forward selection
selected_features_forward_indices = sfs_forward.k_feature_idx_
selected_features_forward_names = [num_var[i] for i in selected_features_forward_indices]
print(f'Selected features (Forward): {selected_features_forward_names}')

# Modeling - Target : 'yield_usda_1000ha' - Selected Features

In [ ]:
num_var = ['area_harvested_usda_1000ha', 'production_usda_1000ha',
           'area_harvested_fao_1000ha', 'production_fao_1000ha',
           'soil_temp_L1_C', 'soil_temp_L2_C', 'soil_temp_L3_C',
           'soil_temp_L4_C', 'temp_C', 'precipitation_era5_mm',
           'wind_northward_m_s', 'soil_water_L2_fraction',
           'soil_water_L4_fraction', 'precipitation_chirps_mm']


cat_var = []

y = df['yield_usda_1000ha']

X = df[cat_var + num_var]
y = pd.DataFrame(y, columns=['yield_usda_1000ha'])

In [ ]:
X.info()

In [ ]:
# Train-test split:

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42 , shuffle=True)

print(f"X_train.shape: {X_train.shape}")
print(f"X_test.shape: {X_test.shape}")
print(f"y_train.shape: {y_train.shape}")
print(f"y_test.shape: {y_test.shape}")

In [ ]:
# Preprocessing with pipelines

from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import MinMaxScaler, OneHotEncoder


pipeline_categorical = Pipeline([('onehot', OneHotEncoder(handle_unknown="ignore")),])

pipeline_numerical = Pipeline([('scaler', MinMaxScaler(feature_range=(0,1))),])

pipeline_full = ColumnTransformer([("categorical", pipeline_categorical, cat_var),
                                   ("numerical", pipeline_numerical, num_var)])

In [ ]:
# Apply the pipeline

pipeline_full.fit(X_train)

X_train_transformed = pipeline_full.transform(X_train)
X_test_transformed = pipeline_full.transform(X_test)

print(f"X_train_transformed.shape: {X_train_transformed.shape}")
print(f"X_test_transformed.shape: {X_test_transformed.shape}")
print()
print('X_train_transformed:', type(X_train_transformed))
print('X_test_transformed:', type(X_test_transformed))

In [ ]:
models = {
    'LinearRegression': LinearRegression(),
    'LassoRegressor': Lasso(max_iter=10000, random_state=42),
    'RidgeRegressor': Ridge(max_iter=10000, random_state=42),
    "ElasticNet": ElasticNet(alpha=0.02, random_state=42),
    # "KNN": KNeighborsRegressor(),
    'DTR': DecisionTreeRegressor(random_state=42),
    'SVR': SVR(),
    'RFR': RandomForestRegressor(random_state=42),
    "ExtraTrees": ExtraTreesRegressor(n_estimators=5, random_state=42),
    'ADABoost': AdaBoostRegressor(random_state=42),
    'XGBoost': xgb.XGBRegressor(random_state=42),
    'CatBoost': CatBoostRegressor(random_state=42),
    'LightGBM': lgb.LGBMRegressor(random_state=42)
}

In [ ]:
# Train multiple models

model_list=[]
r2_list=[]
mae_list=[]
mse_list=[]
rmse_list=[]

for i in range(len(list(models))):
    model=list(models.values())[i]
    model.fit(X_train_transformed,y_train)

    #Make Predictions
    y_pred=model.predict(X_test_transformed)

    # Evaluation Metric:
    #r2 = model.score(X_test_transformed, y_test)
    r2 = r2_score(y_test, y_pred)
    mae = mean_absolute_error(y_test, y_pred)
    mse = mean_squared_error(y_test, y_pred)
    rmse= np.sqrt(mean_squared_error(y_test, y_pred))

    model_list.append(list(models.keys())[i])
    r2_list.append(r2)
    mae_list.append(mae)
    mse_list.append(mse)
    rmse_list.append(rmse)

    # Print the performance metrics for each model
    print(f"Model: {list(models.keys())[i]}")
    print(f"R2 Score: {r2}")
    print(f"MAE: {mae}")
    print(f"MSE: {mse}")
    print(f"RMSE: {rmse}")
    print("="*30)

In [ ]:
# Find the index of the model with the best r2 score
best_index_r2 = r2_list.index(max(r2_list))

# Find the index of the model with the best mae score
best_index_mae = mae_list.index(min(mae_list))

# Find the index of the model with the best mse score
best_index_mse = mse_list.index(min(mse_list))

# Find the index of the model with the best rmse score
best_index_rmse = rmse_list.index(min(rmse_list))

# Determine the model with the best performance based on the chosen metric
best_r2_model = model_list[best_index_r2]
best_mae_model = model_list[best_index_mae]
best_mse_model = model_list[best_index_mse]
best_rmse_model = model_list[best_index_rmse]

# Print the best models and their corresponding metric
print("The model with the best R2 score is:", best_r2_model)
print("R2 score:", r2_list[best_index_r2])
print("="*40)
print("The model with the best MAE score is:", best_mae_model)
print("MAE:", mae_list[best_index_mae])
print("="*40)
print("The model with the best MSE score is:", best_mse_model)
print("MSE:", mse_list[best_index_mse])
print("="*40)
print("The model with the best RMSE score is:", best_rmse_model)
print("RMSE:", rmse_list[best_index_rmse])

In [ ]:
# Create a DF for Result:

results_df = pd.DataFrame({
    'Model': model_list,
    'R2 Score': r2_list,
    'MAE': mae_list,
    'MSE': mse_list,
    'RMSE': rmse_list})

results_df.sort_values(by="RMSE", ascending=True)

# Second Top Model: Ridge Regressor

**Hyperparameter Search for Ridge Regression:**

In [ ]:
# Hyperparameter Search for Ridge Regression:

from sklearn.linear_model import Ridge
from sklearn.model_selection import GridSearchCV

param_grid = {
    'alpha': [0.01, 0.1, 1.0, 10.0],
    'solver': ['auto', 'svd', 'cholesky', 'lsqr', 'sparse_cg', 'sag', 'saga']}

grid_search = GridSearchCV(
    estimator=Ridge(),
    param_grid=param_grid,
    scoring='neg_mean_squared_error',
    cv=5, n_jobs=-1, verbose=1)

grid_search.fit(X_train_transformed, y_train)


In [ ]:
print(grid_search.best_params_)
print("\n",grid_search.best_estimator_)

In [ ]:
Tuned_Ridge_Reg = grid_search.best_estimator_
Tuned_Ridge_Reg

In [ ]:
Tuned_Ridge_Reg .fit(X_train_transformed,y_train)
y_pred=Tuned_Ridge_Reg.predict(X_test_transformed)

In [ ]:
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error

r2 = r2_score(y_test, y_pred)
mae = mean_absolute_error(y_test, y_pred)
mse = mean_squared_error(y_test, y_pred)
rmse = np.sqrt(mse)

print(f"R2 Score: {r2}")
print(f"MAE: {mse}")
print(f"MSE: {mae}")
print(f"RMSE: {rmse}")

In [ ]:
# Calculate residuals
residuals = y_test.values.flatten() - y_pred.flatten()

# Visualize residuals
#sns.kdeplot(residuals)
sns.histplot(residuals, kde=True)
plt.title('Distribution of Residuals - Tuned Ridge Regression')
plt.xlabel('Residuals')
plt.ylabel('Density')
plt.show()

In [ ]:
# Line Plot: Actual vs Predicted Values

plt.plot([y_test.min(), y_test.max()], [y_test.min(), y_test.max()], '--', color='green', label='Actual')
plt.scatter(y_test, y_pred, label='Tuned Ridge Regression Predictions')
plt.xlabel('Actual Values')
plt.ylabel('Predicted Values')
plt.title('Actual vs Predicted Values - Tuned Ridge Regression')
plt.show()

**Ploting the Learning Curve**

In [ ]:
# Ploting the learning curve:

from sklearn.model_selection import learning_curve
train_sizes, train_scores, test_scores = learning_curve(estimator=Tuned_Ridge_Reg,
                                                       X=X_train_transformed, y=y_train,
                                                       cv=5, train_sizes=np.linspace(0.1, 1.0, 20), n_jobs=-1)

plt.figure(figsize=(8,4))
# Calculate training and test mean and std

train_mean = np.mean(train_scores, axis=1)
train_std = np.std(train_scores, axis=1)
test_mean = np.mean(test_scores, axis=1)
test_std = np.std(test_scores, axis=1)

# Plot the learning curve

plt.plot(train_sizes, train_mean, color='blue', marker='o', markersize=5, label='Training Score')
plt.fill_between(train_sizes, train_mean + train_std, train_mean - train_std, alpha=0.15, color='blue')

plt.plot(train_sizes, test_mean, color='green', marker='o', markersize=5, label='Cross-Validation Score')
plt.fill_between(train_sizes, test_mean + test_std, test_mean - test_std, alpha=0.15, color='green')

plt.title('Tuned Ridge Regression Learning Curve ')
plt.xlabel('Training Example', fontsize=12)
plt.ylabel('Accuracy Score', fontsize=12)

plt.legend(loc='lower center', fontsize=8)
plt.show()
